In [1]:
import os
from pathlib import Path
import argparse
import functools

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from sklearn.metrics import auc, roc_curve
import pandas as pd



In [2]:
# Set the display option to show all rows
pd.set_option('display.max_rows', 20)

In [3]:
savedir = "exp/cifar10"

In [12]:
os.listdir(savedir)[20]

'12'

In [13]:
os.listdir(savedir)[42]

'28'

In [16]:
def load_data():

    global keep, logits, scores
    keep = []
    logits = []
    scores = []
    
    for path in os.listdir(savedir):
        # print("path: ", path)
        keep.append(np.load(os.path.join(savedir, path, "keep.npy")))
        logits.append(np.load(os.path.join(savedir, path, "logits.npy")))
        scores.append(np.load(os.path.join(savedir, path, "scores.npy")))

    keep = np.array(keep)
    logits = np.array(logits)
    scores = np.array(scores)

    return keep, logits, scores

In [17]:
keep, logits, scores = load_data()

In [6]:
print(keep.shape)
# print(keep)

(64, 40000)


In [18]:
print(logits.shape)
# print(logits)

(64, 40000, 2, 10)


In [19]:
logits[20]

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [102]:
logits[42]

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [98]:
print(scores.shape)
print(scores)

(64, 40000, 2)
[[[ 4.68748243  4.20020984]
  [-0.45901215  0.70122978]
  [ 2.81220677 -2.70628857]
  ...
  [17.57408614 12.67815007]
  [ 8.61777702  9.48656081]
  [ 9.26889597  9.32298001]]

 [[ 4.59428779  3.78349585]
  [ 7.88184895  8.58197691]
  [-5.24243955 -4.48598317]
  ...
  [20.99291352 15.09406965]
  [ 9.99725349 10.53718927]
  [ 8.8253195   9.62050787]]

 [[ 4.18130685  3.96536509]
  [ 1.93927562  3.94789297]
  [-2.10935169 -4.29676738]
  ...
  [18.23606657 12.97001684]
  [ 8.36554254  8.68055632]
  [ 5.44392951  7.83008651]]

 ...

 [[ 5.66482971  6.94895201]
  [ 2.92638753  3.21772178]
  [-2.85923694 -5.77564323]
  ...
  [18.8793064   9.93001081]
  [ 8.80272302  8.61339657]
  [ 0.56995122  4.78953952]]

 [[ 7.61774112  7.42916475]
  [ 2.4349774   6.41164741]
  [ 5.89888986  3.65823136]
  ...
  [28.13464609 17.59432502]
  [11.62014926 11.80685419]
  [ 8.95643127  8.90055777]]

 [[ 6.03888275  1.82966012]
  [ 5.65124777  5.78665014]
  [ 0.25057937  2.78486696]
  ...
  [16.228

In [99]:
scores[20]

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [100]:
scores[42]

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [18]:
def sweep(score, x):
    """
    Compute a ROC curve and then return the FPR, TPR, AUC, and ACC.
    """
    fpr, tpr, _ = roc_curve(x, -score)
    acc = np.max(1 - (fpr + (1 - tpr)) / 2)
    return fpr, tpr, auc(fpr, tpr), acc

In [28]:
def load_data():
    """
    Load our saved scores and then put them into a big matrix.
    """
    global scores, keep
    scores = []
    keep = []

    for path in os.listdir(savedir):
        scores.append(np.load(os.path.join(savedir, path, "scores.npy")))
        keep.append(np.load(os.path.join(savedir, path, "keep.npy")))
    scores = np.array(scores)
    keep = np.array(keep)

    return scores, keep

In [29]:
load_data()

(array([[[ 4.68748243,  4.20020984],
         [-0.45901215,  0.70122978],
         [ 2.81220677, -2.70628857],
         ...,
         [17.57408614, 12.67815007],
         [ 8.61777702,  9.48656081],
         [ 9.26889597,  9.32298001]],
 
        [[ 4.59428779,  3.78349585],
         [ 7.88184895,  8.58197691],
         [-5.24243955, -4.48598317],
         ...,
         [20.99291352, 15.09406965],
         [ 9.99725349, 10.53718927],
         [ 8.8253195 ,  9.62050787]],
 
        [[ 4.18130685,  3.96536509],
         [ 1.93927562,  3.94789297],
         [-2.10935169, -4.29676738],
         ...,
         [18.23606657, 12.97001684],
         [ 8.36554254,  8.68055632],
         [ 5.44392951,  7.83008651]],
 
        ...,
 
        [[ 5.66482971,  6.94895201],
         [ 2.92638753,  3.21772178],
         [-2.85923694, -5.77564323],
         ...,
         [18.8793064 ,  9.93001081],
         [ 8.80272302,  8.61339657],
         [ 0.56995122,  4.78953952]],
 
        [[ 7.61774112,  7.429

In [72]:
scores.shape

(64, 40000, 2)

In [31]:
keep.shape

(64, 40000)

In [80]:
np.where(np.isnan(scores))

(array([20, 20, 20, ..., 42, 42, 42]),
 array([    0,     0,     1, ..., 39998, 39999, 39999]),
 array([0, 1, 0, ..., 1, 0, 1]))

In [91]:
scores[0, 39999, 1]

9.32298000711081

In [88]:
len(np.where(np.isnan(scores))[0])

160000

In [86]:
np.unique(np.where(np.isnan(scores))[0])

array([20, 42])

In [90]:
np.unique(np.where(np.isnan(scores))[1])[:100]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [52]:
def generate_ours(keep, scores, check_keep, check_scores, in_size=100000, out_size=100000, fix_variance=False):
    """
    Fit a two predictive models using keep and scores in order to predict
    if the examples in check_scores were training data or not, using the
    ground truth answer from check_keep.
    """
    dat_in = []
    dat_out = []


    # iterate data points (x, y).
    for j in range(scores.shape[1]):
        dat_in.append(scores[keep[:, j], j, :])
        dat_out.append(scores[~keep[:, j], j, :])

    in_size = min(min(map(len, dat_in)), in_size)
    out_size = min(min(map(len, dat_out)), out_size)

    dat_in = np.array([x[:in_size] for x in dat_in])
    dat_out = np.array([x[:out_size] for x in dat_out])

    mean_in = np.median(dat_in, 1)
    mean_out = np.median(dat_out, 1)
    
    if fix_variance:
        std_in = np.std(dat_in)
        std_out = np.std(dat_in)
    else:
        std_in = np.std(dat_in, 1)
        std_out = np.std(dat_out, 1)

    print("dat_in: ", dat_in.shape)
    print("dat_out: ", dat_out.shape)

    print("mean_in: ", mean_in.shape)
    print(mean_in)
    print("mean_out: ", mean_out.shape)
    print(mean_out)
    print("std_in: ", std_in.shape)
    print(std_in)
    print("std_out: ", std_out)
    print(std_out)
    

    prediction = []
    answers = []
    for ans, sc in zip(check_keep, check_scores):
        pr_in = -scipy.stats.norm.logpdf(sc, mean_in, std_in + 1e-30)
        pr_out = -scipy.stats.norm.logpdf(sc, mean_out, std_out + 1e-30)
        score = pr_in - pr_out

        prediction.extend(score.mean(1))
        answers.extend(ans)

    return prediction, answers, "", dat_in, dat_out, mean_in, mean_out, std_in, std_out

In [53]:
def do_plot(fn, keep, scores, ntest, legend="", metric="auc", sweep_fn=sweep, **plot_kwargs):
    """
    Generate the ROC curves by using ntest models as test models and the rest to train.
    """

    prediction, answers = fn(keep[:-ntest], scores[:-ntest], keep[-ntest:], scores[-ntest:])

    print("prediction: ")
    print(prediction)
    print("answers: ")
    print(answers)

    fpr, tpr, auc, acc = sweep_fn(np.array(prediction), np.array(answers, dtype=bool))

    low = tpr[np.where(fpr < 0.001)[0][-1]]

    print("Attack %s   AUC %.4f, Accuracy %.4f, TPR@0.1%%FPR of %.4f" % (legend, auc, acc, low))

    metric_text = ""
    if metric == "auc":
        metric_text = "auc=%.3f" % auc
    elif metric == "acc":
        metric_text = "acc=%.3f" % acc

    plt.plot(fpr, tpr, label=legend + metric_text, **plot_kwargs)
    return (acc, auc)


def fig_fpr_tpr():
    plt.figure(figsize=(4, 3))

    do_plot(generate_ours, keep, scores, 1, "Ours (online)\n", metric="auc")

    # do_plot(functools.partial(generate_ours, fix_variance=True), keep, scores, 1, "Ours (online, fixed variance)\n", metric="auc")
    #
    # do_plot(functools.partial(generate_ours_offline), keep, scores, 1, "Ours (offline)\n", metric="auc")
    #
    # do_plot(functools.partial(generate_ours_offline, fix_variance=True), keep, scores, 1, "Ours (offline, fixed variance)\n", metric="auc")
    #
    # do_plot(generate_global, keep, scores, 1, "Global threshold\n", metric="auc")

    plt.semilogx()
    plt.semilogy()
    plt.xlim(1e-5, 1)
    plt.ylim(1e-5, 1)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.plot([0, 1], [0, 1], ls="--", color="gray")
    plt.subplots_adjust(bottom=0.18, left=0.18, top=0.96, right=0.96)
    plt.legend(fontsize=8)
    plt.savefig("fprtpr.png")
    plt.show()

In [54]:
fig_fpr_tpr()

dat_in:  (40000, 31, 2)
dat_out:  (40000, 31, 2)
mean_in:  (40000, 2)
[[ 5.66866874  5.85076141]
 [        nan         nan]
 [ 1.86377865  2.83813249]
 ...
 [22.7923407  13.18756812]
 [        nan         nan]
 [        nan         nan]]
mean_out:  (40000, 2)
[[       nan        nan]
 [0.71230373 2.481409  ]
 [       nan        nan]
 ...
 [       nan        nan]
 [8.52784371 9.15922596]
 [       nan        nan]]
std_in:  (40000, 2)
[[1.31560762 1.35229173]
 [       nan        nan]
 [2.13401356 2.42591937]
 ...
 [5.16536262 3.02299995]
 [       nan        nan]
 [       nan        nan]]
std_out:  [[       nan        nan]
 [2.64778163 2.72025547]
 [       nan        nan]
 ...
 [       nan        nan]
 [1.54028234 1.53659503]
 [       nan        nan]]
[[       nan        nan]
 [2.64778163 2.72025547]
 [       nan        nan]
 ...
 [       nan        nan]
 [1.54028234 1.53659503]
 [       nan        nan]]


ValueError: too many values to unpack (expected 2)

<Figure size 400x300 with 0 Axes>

In [55]:
ntest = 1
prediction, answers, _, dat_in, dat_out, mean_in, mean_out, std_in, std_out = generate_ours(keep[:-ntest], scores[:-ntest], keep[-ntest:], scores[-ntest:])

dat_in:  (40000, 31, 2)
dat_out:  (40000, 31, 2)
mean_in:  (40000, 2)
[[ 5.66866874  5.85076141]
 [        nan         nan]
 [ 1.86377865  2.83813249]
 ...
 [22.7923407  13.18756812]
 [        nan         nan]
 [        nan         nan]]
mean_out:  (40000, 2)
[[       nan        nan]
 [0.71230373 2.481409  ]
 [       nan        nan]
 ...
 [       nan        nan]
 [8.52784371 9.15922596]
 [       nan        nan]]
std_in:  (40000, 2)
[[1.31560762 1.35229173]
 [       nan        nan]
 [2.13401356 2.42591937]
 ...
 [5.16536262 3.02299995]
 [       nan        nan]
 [       nan        nan]]
std_out:  [[       nan        nan]
 [2.64778163 2.72025547]
 [       nan        nan]
 ...
 [       nan        nan]
 [1.54028234 1.53659503]
 [       nan        nan]]
[[       nan        nan]
 [2.64778163 2.72025547]
 [       nan        nan]
 ...
 [       nan        nan]
 [1.54028234 1.53659503]
 [       nan        nan]]


In [69]:
dat_in[0:2,:,:]

array([[[4.18130685, 3.96536509],
        [5.78978207, 6.53505487],
        [5.34519277, 5.01582293],
        [5.57926067, 4.38851879],
        [6.78282791, 4.8421017 ],
        [8.13782555, 8.10384352],
        [5.60239822, 6.01940643],
        [3.78153687, 8.62704514],
        [5.92789557, 4.12296449],
        [5.75109929, 5.41256243],
        [4.69800806, 6.18426652],
        [7.58572041, 5.71451709],
        [5.66866874, 6.20346905],
        [4.77776388, 4.64118995],
        [6.28457681, 3.89888339],
        [4.93736043, 3.9307628 ],
        [7.90428352, 6.96183365],
        [7.1187274 , 7.32674165],
        [4.93377814, 6.05618782],
        [7.56999463, 6.68586856],
        [6.65928525, 6.63802794],
        [5.99501771, 5.45824855],
        [5.05765442, 5.85076141],
        [5.38011838, 6.57334407],
        [3.78042647, 2.57286658],
        [5.88999395, 4.1591786 ],
        [4.37914161, 4.66806034],
        [2.18455064, 5.60448683],
        [5.85974345, 6.16879199],
        [5.664

In [65]:
np.median(dat_in[:,:,:], 1)

array([[ 5.67674875,  5.21419268],
       [        nan,         nan],
       [ 2.22219514,  2.22789994],
       ...,
       [23.12812907, 13.62188134],
       [ 9.10877341,  9.37971657],
       [ 8.45368755,  8.87077046]])

In [ ]:
np.median(dat_in[:,0:10,:], 1)

In [50]:
in_size = 100000
min(min(map(len, dat_in)), in_size)

31

In [51]:
map(len, dat_in)